In [14]:
import psutil
import json
import xml.etree.ElementTree as ET
import zipfile
import os

### Системная информация

In [21]:
def system_information():
    partitions = psutil.disk_partitions()

    for partition in partitions:
        print("Имя диска:", partition.device)
        print("Метка тома:", partition.mountpoint)

        try:
            disk_usage = psutil.disk_usage(partition.mountpoint)
            print("Размер диска:", convert_bytes(disk_usage.total))
            print("Тип файловой системы:", partition.fstype)
        except PermissionError:
            print("Нет доступа к информации о диске")

        print()

def convert_bytes(size, precision=2):
    suffixes = ['B', 'KB', 'MB', 'GB', 'TB']
    index = 0
    while size >= 1024 and index < len(suffixes) - 1:
        size /= 1024
        index += 1
    return f"{size:.{precision}f} {suffixes[index]}"

### Работа с файлами

In [22]:
def create_file(file):
    if os.path.exists(file):
        print("Файл с таким именем уже существует.")
    else:
        with open(file, 'w') as f:
            pass
        print("Файл успешно создан.")

def write_to_file(file, data):
    try:        
        if not os.path.exists(file):
            print("Файл не найден.")
            return
        
        with open(file, 'w') as f:
            f.write(data)
        print("Файл успешно записан.")
        
    except FileNotFoundError:
        print("Файл не найден.")

def read_file(file):
    try:
        with open(file, 'r') as f:
            data = f.read()
        print("Содержимое файла:")
        print(data)
    except FileNotFoundError:
        print("Файл не найден.")

def delete_file(file):
    try:
        os.remove(file)
        print("Файл успешно удален.")
    except FileNotFoundError:
        print("Файл не найден.")

### Работа с форматом JSON

In [23]:
def create_json_file(file):
    data = {}
    if os.path.exists(file):
        print("Файл с таким именем уже существует.")
    else:
        with open(file, 'w') as f:
            json.dump(data, f)
        print("Файл JSON успешно создан.")

def add_data_json_file(file, data):
    try:
        if not os.path.exists(file):
            print("Файл не найден.")
            return
        
        data = json.loads(data)
        with open(file, "w") as f:
            json.dump(data, f)
        print("Объект успешно записан в файл.")
        
    except FileNotFoundError:
        print("Файл не найден.")
        
    except json.JSONDecodeError:
        print("Ошибка: Неправильный формат данных. Введите данные в формате JSON.")
        print("Пример данных для успешной записи в формат JSON:")
        print('{"key1": "value1", "key2": 123, "key3": ["item1", "item2", "item3"]}')

def read_json_file(file):
    try:
        with open(file, 'r') as f:
            data = json.load(f)
        print("Содержимое файла JSON:")
        print(data)
    except FileNotFoundError:
        print("Файл не найден.")

def delete_json_file(file):
    try:
        os.remove(file)
        print("Файл JSON успешно удален.")
    except FileNotFoundError:
        print("Файл не найден.")

### Работа с форматом XML

In [24]:
def create_xml_file(file):
    try:
        if os.path.exists(file):
            raise FileExistsError("Файл уже существует.")
        else:
            root_tag = input("Введите имя корневого элемента: ")
            root = ET.Element(root_tag)
            tree = ET.ElementTree(root)
            tree.write(file)
            print("Файл XML успешно создан.")

    except FileExistsError as e:
        print("Ошибка:", str(e))
    except Exception as e:
        print("Ошибка при создании файла XML:", str(e))
    
def write_to_xml_file(file, root):
    try:
        if not os.path.exists(file):
            print("Файл не найден.")
            return
        else:
            tree = ET.parse(file)
            root = tree.getroot()
            child_tag = input("Введите имя дочернего элемента: ")
            child_text = input("Введите текст для дочернего элемента: ")
            child = ET.Element(child_tag)
            child.text = child_text
            root.append(child)
            tree.write(file)
            print("Данные успешно записаны в файл формата XML.")
        
    except FileNotFoundError:
        print("Файл не найден.")

def read_xml_file(file):
    try:
        tree = ET.parse(file)
        root = tree.getroot()
        print("Содержимое файла XML:")
        for child in root:
            print(child.tag, child.text)
    except FileNotFoundError:
        print("Файл не найден.")

def delete_xml_file(file):
    try:
        os.remove(file)
        print("Файл XML успешно удален.")
    except FileNotFoundError:
        print("Файл не найден.")
        
root = None

### Работа с zip архивом

In [25]:
def create_zip_archive(archive):
    try:
        if os.path.exists(archive):
            raise FileExistsError("Файл уже существует.")
            
        else:
            zip_archive = zipfile.ZipFile(archive, 'w')
            print("Архив успешно создан.")
            return zip_archive
    
    except FileExistsError as e:
        print("Ошибка:", str(e))
    except Exception as e:
        print("Ошибка при создании архива:", str(e))


def add_file_to_archive(archive, file):
    try:
        if not os.path.exists(archive):
            print("Архив не найден.")
            return
        else:
            with zipfile.ZipFile(archive, 'a') as zip_archive:
                zip_archive.write(file)
            print("Файл успешно добавлен в архив.")
            
    except FileNotFoundError:
        print("Архив не найден.")

def extract_file_from_archive(archive):
    try:
        if not os.path.exists(archive):
            print("Архив не найден.")
            return
        
        with zipfile.ZipFile(archive, 'r') as zip_archive:
            file_list = zip_archive.namelist()
            print("Содержимое архива:")
            for i, file in enumerate(file_list):
                print(f"{i+1}. {file}")
            
            choice = input("Введите номер файла для разархивации (или 'q' для выхода): ")
            if choice == 'q':
                return
        
            try:
                file_index = int(choice) - 1
                selected_file = file_list[file_index]
                zip_archive.extract(selected_file)
                print("Файл успешно разархивирован.")
                
                print("Содержимое файла:")
                with open(selected_file, 'r') as f:
                    data = f.read()
                print(data)

            except (ValueError, IndexError):
                print("Некорректный выбор файла.")
    except FileNotFoundError:
        print("Архив не найден.")

def delete_archive(archive):
    choice = input("Хотите удалить только файл в архиве или весь архив? Введите 'файл' или 'архив': ")
    if choice == 'файл':
        file_name = input("Введите название файла для удаления: ")
        try:
            with zipfile.ZipFile(archive, 'w') as zip_archive:
                file_list = zip_archive.namelist()
                if file_name in file_list:
                    zip_archive.extract(file_name)
                    print("Файл успешно удален из архива.")
                else:
                    print("Файл не найден в архиве.")
        except FileNotFoundError:
            print("Архив не найден.")
    elif choice == 'архив':
        try:
            os.remove(archive)
            print("Архив успешно удален.")
        except FileNotFoundError:
            print("Архив не найден.")
    else:
        print("Некорректный выбор.")


## Программа

In [26]:
flag = True
while flag:
    print("Выберите действие:")
    print("1. Вывести информацию о логических дисках")
    print("2. Работа с файлами")
    print("3. Работа с форматом JSON")
    print("4. Работа с форматом XML")
    print("5. Работа с zip архивом")
    print("6. Выйти из программы")

    action = input("Введите номер действия: ")

    if action == "1":
        system_information()
        
    elif action == "2":
        while True:
            print("Что вы хотите сделать с файлом?")
            print("1. Создать файл")
            print("2. Записать в файл данные")
            print("3. Прочитать файл")
            print("4. Удалить файл")
            print("5. Выйти")

            choice = input("Введите номер действия:")

            if choice == "1":
                file = input("Введите название файла: ")
                create_file(file)
            elif choice == "2":
                file = input("Введите название файла: ")
                data = input("Введите строку:")
                write_to_file(file, data)
            elif choice == "3":
                file = input("Введите название файла: ")
                read_file(file)
            elif choice == "4":
                file = input("Введите название файла: ")
                delete_file(file)
            elif choice == "5":
                print("Выход из программы.")
                break
            else:
                print("Некорректный выбор. Попробуйте еще раз.")
    
    elif action == "3":
        while True:
            print("Что вы хотите сделать в формате JSON?")
            print("1. Создать файл в формате JSON")
            print("2. Создать новый объект и запись его в файл")
            print("3. Прочитать файл JSON")
            print("4. Удалить файл JSON")
            print("5. Выйти")

            choice = input("Введите номер действия:")

            if choice == "1":
                file = input("Введите название файла: ")
                create_json_file(file)
            elif choice == "2":
                file = input("Введите название файла: ")
                data = input("Введите данные в формате JSON: ")
                add_data_json_file(file, data)
            elif choice == "3":
                file = input("Введите название файла: ")
                read_json_file(file)
            elif choice == "4":
                file = input("Введите название файла: ")
                delete_json_file(file)
            elif choice == "5":
                print("Выход из программы.")
                break
            else:
                print("Некорректный выбор. Попробуйте еще раз.")
                
    elif action == "4":

        while True:
            print("Что вы хотите сделать в формате XML?")
            print("1. Создать файл в формате XML")
            print("2. Записать в файл новые данные")
            print("3. Прочитать файл XML")
            print("4. Удалить файл XML")
            print("5. Выйти")

            choice = input("Введите номер действия:")

            if choice == "1":
                file = input("Введите название файла: ")
                create_xml_file(file)
                root = ET.ElementTree(file).getroot()
            elif choice == "2":
                if root is None:
                    print("Сначала создайте файл XML.")
                else:
                    file = input("Введите название файла: ")
                    write_to_xml_file(file, root)
            elif choice == "3":
                file = input("Введите название файла: ")
                read_xml_file(file)
            elif choice == "4":
                file = input("Введите название файла: ")
                delete_xml_file(file)
            elif choice == "5":
                print("Выход из программы.")
                break
            else:
                print("Некорректный выбор. Попробуйте еще раз.")
                    
    elif action == "5":
         while True:
            print("Что вы хотите сделать с zip архивом?")
            print("1. Создать архив в формате zip")
            print("2. Добавить файл в архив zip")
            print("3. Разархивировать файл и вывести данные о нем")
            print("4. Удалить файл и архив zip")
            print("5. Выйти")

            choice = input("Введите номер действия:")

            if choice == "1":
                archive = input("Введите название архива: ")
                create_zip_archive(archive)
            elif choice == "2":
                archive = input("Введите название архива: ")
                file = input("Введите название файла: ")
                add_file_to_archive(archive, file)
            elif choice == "3":
                archive = input("Введите название архива: ")
                extract_file_from_archive(archive)
            elif choice == "4":
                archive = input("Введите название архива: ")
                delete_archive(archive)
            elif choice == "5":
                print("Выход из программы.")
                break
            else:
                print("Некорректный выбор. Попробуйте еще раз.")
                    
    elif action == "6":
        print("Выход из программы.")
        flag = False
    else:
        print("Некорректный выбор. Попробуйте еще раз.")

Выберите действие:
1. Вывести информацию о логических дисках
2. Работа с файлами
3. Работа с форматом JSON
4. Работа с форматом XML
5. Работа с zip архивом
6. Выйти из программы
Введите номер действия: 1
Имя диска: C:\
Метка тома: C:\
Размер диска: 120.00 GB
Тип файловой системы: NTFS

Имя диска: D:\
Метка тома: D:\
Размер диска: 341.33 GB
Тип файловой системы: NTFS

Выберите действие:
1. Вывести информацию о логических дисках
2. Работа с файлами
3. Работа с форматом JSON
4. Работа с форматом XML
5. Работа с zip архивом
6. Выйти из программы
Введите номер действия: 6
Выход из программы.
